Adapted from https://github.com/astudillo20lca/hypercognitive

**Conference Notebook Kernel: `bw25`**

In [1]:
from pathlib import Path
from fs.zipfs import ZipFS

import numpy as np
import pandas as pd

import bw_processing
import bw2calc as bwc

## Run our minimalistic

Lets run our minimalistic LCA with the datapackages that we've just created. No need for projects, no need for `bw2data` at all

## locate the datapackages

a dictionary with the filename as key and the path as value

In [2]:
datapackges = {path.stem.split('.')[0]:path for path in Path.cwd().glob('*.zip')}

In [3]:
datapackges.keys()

dict_keys(['ipccs', 'testdb', 'mini_biosphere'])

In [4]:
# data package with impact assessment method
ipcc_dp = bw_processing.load_datapackage(ZipFS(datapackges['ipccs']),proxy=True)
# data package with minimal biosphere database
biosphere_dp = bw_processing.load_datapackage(ZipFS(datapackges['mini_biosphere']),proxy=True)
# data package with foreground database
test_dp = bw_processing.load_datapackage(ZipFS(datapackges['testdb']),proxy=True)

## get some medatadata 

we want to be able to recover some of the metadata, for example the name of the activities. This is part of our datapackage

In [5]:
metadata_foreground,_ = test_dp.get_resource('testdb_activity_metadata')

In [6]:
metadata_foreground

,reference product,unit,name,location,id
0,NaN,kilogram,my_machine,NaN,23995


In [7]:
metadata_biosphere,_ = biosphere_dp.get_resource('mini_biosphere_activity_metadata')

In [8]:
metadata_biosphere

,reference product,unit,name,location,id
0,NaN,kilogram,"Carbon dioxide, fossil",GLO,23993
1,NaN,kilogram,"Methane, fossil",GLO,23994


Dictionary with the ids of the different nodes in my databases

In [9]:
name_id_dict = { 
    **metadata_foreground.set_index('name')['id'].to_dict(),
    **metadata_biosphere.set_index('name')['id'].to_dict()
}
name_id_dict

{'my_machine': 23995,
 'Carbon dioxide, fossil': 23993,
 'Methane, fossil': 23994}

now let's run the same calculation as in the previous notebook to make sure we get the same results

In [10]:
lca = bwc.LCA(demand={name_id_dict['my_machine']:1},data_objs=[ipcc_dp,biosphere_dp,test_dp])
lca.lci()
lca.lcia()

In [11]:
lca.score

39.70000076293945

:)

## Dynamic LCA with Brightway 2.5

Now imagine we get measurements for the CO2 and CH4 emissions of activity _my_machine_. In the HyperCOG project these will come from `aquisition` nodes.

they could be deserialized as json file that can be easily :

In [21]:
emission_measurements = {
('Carbon dioxide, fossil','my_machine'):[20,30.5,10,0],
('Methane, fossil','my_machine'):[0,2,3,4]
}

In [22]:
emission_measurements_df = pd.DataFrame(emission_measurements)
emission_measurements_df

,"Carbon dioxide, fossil","Methane, fossil"
,my_machine,my_machine
0,20.0,0
1,30.5,2
2,10.0,3
3,0.0,4


In [23]:
lca.biosphere_matrix.toarray()

array([[10.],
       [ 1.]])

esentially what we want is a series of calculations where the values of the biosphere matrix are replaced by our new stream of data

In [24]:
indices = [(name_id_dict[substance],name_id_dict[activity]) for substance,activity in emission_measurements_df.columns]
indices

[(23993, 23995), (23994, 23995)]

In [25]:
indices_array = np.array(indices, dtype = bw_processing.INDICES_DTYPE)

In [31]:
emissions_iterator = (row for row in emission_measurements_df.values)

updating the LCA with a dynamic vector

In [32]:
overwriter = bw_processing.create_datapackage()

overwriter.add_dynamic_vector(
    matrix="biosphere_matrix",
    interface=emissions_iterator,
    indices_array=indices_array,
)

In [33]:
lca = bwc.LCA(demand={name_id_dict['my_machine']: 1}, data_objs=[ipcc_dp,biosphere_dp,test_dp,overwriter])
lca.lci()
lca.lcia()
lca.score

20.0

In [34]:
for _ in range(len(emission_measurements_df)):
    try:
        next(lca)
        print(f"updated score: {lca.score:.2f}")
    except StopIteration:
        print('sequence completed')
        break

updated score: 89.90
updated score: 99.10
updated score: 118.80
sequence completed
